<a href="https://colab.research.google.com/github/c3Zor/jupyter_lab/blob/master/signs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pwd

In [0]:
cd sample_data

/content/sample_data


In [0]:
!ls

anscombe.json		      mnist_test.csv
california_housing_test.csv   mnist_train_small.csv
california_housing_train.csv  README.md


# New Section

In [7]:
#douwnloading data from bitbucket
!git clone https://bitbucket.org/jadslim/german-traffic-signs

Cloning into 'german-traffic-signs'...
remote: Counting objects: 6, done.
remote: Compressing objects: 100% (6/6), done.
remote: Total 6 (delta 0), reused 0 (delta 0)
Unpacking objects: 100% (6/6), done.


In [9]:
!ls german-traffic-signs/

signnames.csv  test.p  train.p	valid.p


In [0]:
#importing libraries for signs
import numpy as np
import matplotlib.pyplot as plt
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers.convolutional import Conv2D, MaxPooling2D 
from keras.optimizers import Adam
from keras.utils.np_utils import to_categorical
#for unpickle files in repository
import pickle

In [0]:
#random seed to generate data allways the same
np.random.seed(0)

In [0]:
# with opens and closes the file in this case we open train.p, second argument is 
# what to do with the data opening in binary mode for reading ar writing 
with open('german-traffic-signs/train.p', 'rb') as f:
  train_data = pickle.load(f)
# doing the same  for val data
with open('german-traffic-signs/valid.p', 'rb') as f:
  val_data = pickle.load(f)
# doing the same for test data
with open('german-traffic-signs/test.p', 'rb') as f:
  test_data = pickle.load(f)

In [0]:
# out data is in dictiondary so we split it in two dictionaries
x_train, y_train = train_data['features'], train_data['labels']
x_val, y_val = val_data['features'], val_data['labels']
x_test, y_test = test_data['features'], test_data['labels']